In [ ]:
import os
import sys
import polars as pl
import pandas as pd#导入csv文件的库
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
import kaggle_evaluation.mcts_inference_server
import numpy as np#对矩阵进行科学计算的库
import re#用于正则表达式提取的库
import gc#垃圾回收的库
import sklearn
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV  #模型参数调优小参数 
import optuna   #模型参数调优多参数 
from sklearn.preprocessing import StandardScaler
import sklearn.ensemble
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedGroupKFold
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import hstack, csr_matrix
from sklearn.feature_selection import SelectKBest, chi2,f_regression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from pathlib import Path
comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')

# **Read Data**

In [ ]:
train=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
# train=train.to_pandas()
print(f"len(train):{len(train)}")
test=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
# test=test.to_pandas()
print(f"len(test):{len(test)}")


In [ ]:
test.head()


# 异常数据判断处理

In [ ]:
# 1.先解决数据质量问题：确保数据干净无误，这是任何机器学习项目的基础。
# 2.接下来重点放在特征工程上：好的特征往往比复杂的模型更能提高性能。
# 3.然后进行模型选择与调参：找到最适合你数据的模型，并优化其超参数。
# 4.如果还有改进空间，可以尝试增加数据量或调整目标变量。

In [ ]:
# # 检查每列的缺失值数量
# missing_values = train.isnull().sum()
# print(missing_values)

In [ ]:
# train.head()

In [ ]:
# target = 'utility_agent1'




# df.describe().T

In [ ]:
# # len(df['GameRulesetName'].unique())
# # df.head()
# df= train.to_pandas()
# df.info()
# #####删除列只有一个数据的无用列
# dorp_unique_less_col=[]
# drop_two_frequency_1_col=[]
# drop_muti_frequency_1_col=[]
# str_col=[]
# one_hot_str_col=[]
# for col in df.drop(['Id','num_draws_agent1','num_losses_agent1',
#                     'num_wins_agent1','utility_agent1'],axis=1).columns:
#     col_unique_count = len(df[col].unique())
# #     print("===col",col_unique_count)
#     if col_unique_count == 1:
# #         print("========col==",df[col].unique())
#         dorp_unique_less_col.append(col)
#     else :
# #          col_value_counts=df[col].value_counts().reset_index()
#          #模型筛选出的无用特征,出现频率少于1%
#          # 检查列是否为二分
#          if set(df[col].unique()) == {0, 1}:
#             # 计算值为1的频率
#             freq_1 = df[col].mean()
#             # 如果频率小于1%，加入删除列表
#             if freq_1 < 0.01:
#                 drop_two_frequency_1_col.append(col)
# #                print("========col==",col,col_value_counts)
#          else :
#                # 检查列是否为数值类型
#                 col_type =df[col].dtype
#                 if pd.api.types.is_numeric_dtype(col_type):
#                  # 计算每个值的出现频率
#                     value_frequencies = df[col].value_counts(normalize=True)
# #                     print("=col==",col,value_frequencies)

#                     # 检查是否有值的频率小于1%
#                     if any(freq < 0.01 for freq in value_frequencies):
#                         drop_muti_frequency_1_col.append(col)
#                 else:
#                      col_value_counts=df[col].value_counts().reset_index()
#                      print(col,len(col_value_counts))
#                      if len(col_value_counts)>50:
#                                   str_col.append(col)
#                      else:
        
#                                 one_hot_str_col.append(col)


# # print(len(dorp_unique_less_col))
# # print(len(drop_two_frequency_1_col))
# # print(len(drop_muti_frequency_1_col))
# print(str_col)
# print("==one_hot_str_col==",one_hot_str_col)
# # train.drop()

                        
    


# **数据分析**

In [ ]:
train.describe()

In [ ]:
# 分类变量的性质：
# 有序变量：使用 LabelEncoder 或 OrdinalEncoder。
# 无序变量：使用 OneHotEncoder。
# 数据类型：
# 文本数据：使用 TfidfVectorizer 或 CountVectorizer。
# 分类标签：使用 LabelEncoder、OrdinalEncoder 或 OneHotEncoder。
# 模型需求：
# 线性模型：通常需要独热编码（OneHotEncoder）来避免顺序关系的误导。
# 树模型：可以使用 LabelEncoder 或 OrdinalEncoder，因为树模型不会受到顺序关系的影响。
# 数据规模：
# 大规模数据：使用 TfidfVectorizer 或 CountVectorizer 生成稀疏矩阵，节省内存。




# 数据处理

In [ ]:
class Preprocessor():
    def __init__(self,seed=2024,target='utility_agent1',train=None,num_folds=3,
                 CV_LB_path="/kaggle/input/mcts-eda-about-cv-and-lb/0917CV_LB.csv"):
        self.seed=seed
        self.target=target
        self.train=train
        self.models=[]#训练和推理的模型
        self.tfidfs=[]#字符串的tfidf模型
        self.num_folds=num_folds
        self.obj_cols =[]  ##字符列
        self.enc=None
        self.dorp_unique_less_col=[]
        self.drop_two_frequency_1_col=[]
        self.drop_muti_frequency_1_col=[]
        self.one_hot_str_col=[]  ##用独热编码的字端
        self.onehot_encoder =None ###one-hot编码
        self.tfidfvectorizer_dict ={}    ### 词向量
        self.tfidfvectorizer_select_col_dict={}
        self.scaler = StandardScaler()
    
        #遍历表格df的所有列修改数据类型减少内存使用
    def reduce_mem_usage(self,df, float16_as32=True):
        #memory_usage()是df每列的内存使用量,sum是对它们求和, B->KB->MB
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

        for col in df.columns:#遍历每列的列名
            col_type = df[col].dtype#列名的type
            if col_type != object and str(col_type)!='category':#不是object也就是说这里处理的是数值类型的变量
                c_min,c_max = df[col].min(),df[col].max() #求出这列的最大值和最小值
                if str(col_type)[:3] == 'int':#如果是int类型的变量,不管是int8,int16,int32还是int64
                    #如果这列的取值范围是在int8的取值范围内,那就对类型进行转换 (-128 到 127)
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    #如果这列的取值范围是在int16的取值范围内,那就对类型进行转换(-32,768 到 32,767)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    #如果这列的取值范围是在int32的取值范围内,那就对类型进行转换(-2,147,483,648到2,147,483,647)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    #如果这列的取值范围是在int64的取值范围内,那就对类型进行转换(-9,223,372,036,854,775,808到9,223,372,036,854,775,807)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:#如果是浮点数类型.
                    #如果数值在float16的取值范围内,如果觉得需要更高精度可以考虑float32
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        if float16_as32:#如果数据需要更高的精度可以选择float32
                            df[col] = df[col].astype(np.float32)
                        else:
                            df[col] = df[col].astype(np.float16)  
                    #如果数值在float32的取值范围内，对它进行类型转换
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    #如果数值在float64的取值范围内，对它进行类型转换
                    else:
                        df[col] = df[col].astype(np.float64)
        #计算一下结束后的内存
        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        #相比一开始的内存减少了百分之多少
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

        return df
        
        
        #清理df的字符串的列
    def clean(self,df,col):
        #字符串缺失值填充
        df[col]=df[col].fillna("nan")
        #字符串转换成小写
        df[col]=df[col].apply(lambda x:x.lower())
        #考虑到这种字符串 ‘MCTS-UCB1-0.6-NST-false‘
        ps='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        for p in ps:
            df[col]=df[col].apply(lambda x:x.replace(p,' '))
        return df
    
#         self.str_col=[]
        #检查CV和LB情况的CV,LB统计表
#         self.check=pd.read_csv(CV_LB_path)
    def make_trian_pro(self,df):
        for col in df.drop(['Id','GameRulesetName','num_draws_agent1','num_losses_agent1',
                            'num_wins_agent1','utility_agent1'],axis=1).columns:
            col_unique_count = len(df[col].unique())
            if col_unique_count == 1:   ##//只有一列值 删除
                self.dorp_unique_less_col.append(col)
            else :
        #          col_value_counts=df[col].value_counts().reset_index()
                 #模型筛选出的无用特征,出现频率少于1%
                 # 检查列是否为二分
                 if set(df[col].unique()) == {0, 1}:
                    # 计算值为1的频率
                    freq_1 = df[col].mean()
                    # 如果频率小于1%，加入删除列表
                    if freq_1 < 0.01:
                         self.drop_two_frequency_1_col.append(col)
        #                print("========col==",col,col_value_counts)
                 else :
                       # 检查列是否为数值类型
                        col_type =df[col].dtype
                        if pd.api.types.is_numeric_dtype(col_type):
                         # 计算每个值的出现频率
                            value_frequencies = df[col].value_counts(normalize=True)
        #                     print("=col==",col,value_frequencies)

                            # 检查是否有值的频率小于1%
                            if any(freq < 0.01 for freq in value_frequencies):
                                 self.drop_muti_frequency_1_col.append(col)
                        else:
                              col_value_counts=df[col].value_counts().reset_index()
#                               print(col_value_counts)
                              if len(col_value_counts)>20:
                                  self.obj_cols.append(col)
                              else:
                                  self.one_hot_str_col.append(col)

    
    def data_deal(self,train,mode='trian'):
        print("agent1 agent2 feature")
        cols=['selection','exploration_const','playout','score_bounds']
        for i in range(len(cols)):
            for j in range(2):
                train[f'{cols[i]}{j+1}']=train[f'agent{j+1}'].apply(lambda x:x.split('-')[i+1])
          #即:rule去掉'(game "',然后找第一个双引号,并用双引号后面的内容在前面加上'('作为新的rule.
        def drop_gamename(rule):
            if rule is None:
                return None  # 或者返回一个默认值，例如空字符串 ''
            rule=rule[len('(game "'):]
            for i in range(len(rule)):
                if rule[i]=='"':
                    return rule[i+1:]
        train['LudRules']=train['LudRules'].apply(lambda x:drop_gamename(x))
        #########
       
        
        
         #删除
        train = train.drop(['agent1','agent2'],axis=1)
        ########
        
        if mode =="train":
            self.make_trian_pro(train)
            cols_to_drop = ["Id",'num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1']
            train = train.drop(cols_to_drop,axis=1)
            train = train.drop(self.dorp_unique_less_col,axis=1)
            train = train.drop(self.drop_two_frequency_1_col,axis=1)
#             train = train.drop(self.drop_muti_frequency_1_col)
#             obj_cols =  train.select(pl.col(pl.String)).columns
            obj_cols = self.obj_cols
            print("=============obj_colslen============",len(self.obj_cols),obj_cols)
            if len(self.one_hot_str_col)>0:
               onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
               self.onehot_encoder =onehot_encoder
               train_encoded = pd.DataFrame(onehot_encoder.fit_transform(train[self.one_hot_str_col])
                                            , columns=onehot_encoder.get_feature_names_out(),
                                         index=train.index)
#                print("==============onehot_encoder============",train_encoded)
               train = pd.concat([train, train_encoded ], axis=1)
               train = train.drop(self.one_hot_str_col,axis=1)
            

                
            ##换成词开里
            if len(obj_cols) >0 :
                for one_col in obj_cols:
                        tfidf_matrices = []

                        tfidfvectorizer = TfidfVectorizer(max_features=600)
                        vector_train = tfidfvectorizer.fit_transform(train[one_col])
                        feature_names = ['_'.join([one_col, name]) for name in tfidfvectorizer.get_feature_names_out()]

                        # 将 TF-IDF 向量转换为 DataFrame
                        vector_train_df = pd.DataFrame.sparse.from_spmatrix(vector_train, columns=feature_names, index=train.index)

                        # 存储 TF-IDF 矩阵和特征名称
                        tfidf_matrices.append(vector_train)


                        # 合并所有文本列的 TF-IDF 向量
                        combined_tfidf_matrix = hstack(tfidf_matrices).tocsr()

                        # 特征选择
                        selector = SelectKBest(f_regression, k=50)
                        selected_features = selector.fit_transform(combined_tfidf_matrix, train[self.target])

                        # 获取选中的特征名称
                        selected_feature_indices = selector.get_support(indices=True)
                        selected_feature_names = [feature_names[i] for i in selected_feature_indices]

                        # 将选中的特征转换为 DataFrame
                        selected_features_df = pd.DataFrame.sparse.from_spmatrix(selected_features, columns=selected_feature_names, index=train.index)

                        # 将选中的特征与原始 DataFrame 合并
                        train = pd.concat([train, selected_features_df], axis=1)

                        # 存储 TfidfVectorizer 和选中的特征名称
                        self.tfidfvectorizer_dict[one_col] = tfidfvectorizer
                        self.tfidfvectorizer_select_col_dict[one_col] = selected_feature_names
        #                     tfidfvectorizer = TfidfVectorizer(max_features=600)
#                     tfidf_matrices = []
#                     all_feature_names = []
#                     vector_train = tfidfvectorizer.fit_transform(train[one_col])
#                     feature_names = ['_'.join([one_col, name]) for name in 
#                                      tfidfvectorizer.get_feature_names_out()]
#                     vector_train = pd.DataFrame.sparse.from_spmatrix(vector_train, columns=feature_names, 
#                                                                      index=train.index)
# #                     train = pd.concat([train, vector_train], axis=1)
#                     # 存储 TF-IDF 矩阵和特征名称
#                     tfidf_matrices.append(vector_train)
#                     all_feature_names.extend(feature_names)
#                     # 合并所有文本列的 TF-IDF 向量
#                     combined_tfidf_matrix = hstack(tfidf_matrices).tocsr()
# #                     combined_tfidf_matrix=tfidf_matrices
#                     # 特征选择
#                     selector = SelectKBest(chi2, k=50)
#                     selected_features = selector.fit_transform(combined_tfidf_matrix, train[self.target])

#                     # 获取选中的特征名称
#                     selected_feature_indices = selector.get_support(indices=True)
#                     selected_feature_names = [all_feature_names[i] for i in selected_feature_indices]

#                     # 将选中的特征转换为 DataFrame
#                     selected_features_df = pd.DataFrame.sparse.from_spmatrix(selected_features, 
#                                                                              columns=selected_feature_names,
#                                                                              index=vector_train.index)
#                     train = train.drop(feature_names,axis=1)

#                     # 将选中的特征与原始 DataFrame 合并
#                     df = pd.concat([train, selected_features_df], axis=1)

#                     self.tfidfvectorizer_dict[one_col] =tfidfvectorizer
#                     self.tfidfvectorizer_select_col_dict[one_col] = selected_feature_names
                
                train = train.drop(obj_cols,axis=1)
                    
                    
                    
#             enc = OrdinalEncoder(
#                 handle_unknown='use_encoded_value', unknown_value=-999, encoded_missing_value=-9999)
#             enc.fit(train[obj_cols])
#             train_transformed = enc.transform(train[obj_cols])
#             for e, c in enumerate(obj_cols):
# #                 train = train.with_columns(pl.Series(c, train_transformed[:, e]))
#                   train[c] = train_transformed[:, e]

            self.obj_cols = obj_cols
#             self.enc = enc
#             return train
        else :
            obj_cols = self.obj_cols
            train = train.drop(self.dorp_unique_less_col,axis=1)
            train = train.drop(self.drop_two_frequency_1_col,axis=1)
            if len(self.one_hot_str_col)>0:
               onehot_encoder=self.onehot_encoder 
               train_encoded = pd.DataFrame(onehot_encoder.transform(train[self.one_hot_str_col])
                                            , columns=onehot_encoder.get_feature_names_out(),
                                         index=train.index)
               train = pd.concat([train, train_encoded ], axis=1)
               train = train.drop(self.one_hot_str_col,axis=1)

#             test =train
#             test_transformed = self.enc.transform(test[obj_cols])
#             for e, c in enumerate(obj_cols):
# #                 test = test.with_columns(pl.Series(c, test_transformed[:, e]))
#                  test[c] = test_transformed[:, e]
            #换成词开里
            if len(obj_cols) >0 :
                for one_col in obj_cols:
                    tfidfvectorizer = self.tfidfvectorizer_dict[one_col]
                    vector_train = tfidfvectorizer.transform(train[one_col])
                    feature_names = ['_'.join([one_col, name]) for name in 
                                     tfidfvectorizer.get_feature_names_out()]
                    vector_train = pd.DataFrame.sparse.from_spmatrix(vector_train, columns=feature_names, 
                                                                     index=train.index)
                    selected_feature_names = self.tfidfvectorizer_select_col_dict[one_col] 
                    train = pd.concat([train, vector_train[selected_feature_names]], axis=1)
      
                train = train.drop(obj_cols,axis=1)
#             return train
        train=self.reduce_mem_usage(train)
        return train
      
        
        
    def RMSE(self,y_true,y_pred):
        return np.sqrt(np.mean((y_true-y_pred)**2))
    def make_model(self):
        seed = self.seed
        
        
        
#         cat_model = CatBoostRegressor(
#         iterations=1500, learning_rate=0.04, depth=9, l2_leaf_reg=0.0005,
#         min_data_in_leaf=25, bagging_temperature=0.04,
#         grow_policy='SymmetricTree', verbose=0,
#             random_state = seed
#         )
#     {'iterations': 1480, 'learning_rate': 0.09333122834440728, 'depth': 8, 
#      'l2_leaf_reg': 0.0021736125373129456, 'min_data_in_leaf': 42,
#      'bagging_temperature': 0.7135945403756551, 'grow_policy': 'SymmetricTree'}.    
    
    
        cat_model = CatBoostRegressor(
        iterations=1480, learning_rate= 0.09333122834440728, depth=8, l2_leaf_reg=0.0021736125373129456,
        min_data_in_leaf=42, bagging_temperature= 0.7135945403756551,
        grow_policy='SymmetricTree', verbose=0,
            random_state = seed
        )
        
#         {'learning_rate': 0.0304874420014571, 'num_leaves': 176,
#          'n_estimators': 1724, 'min_child_samples': 12, 
#          'subsample': 0.8996187429841459, 'colsample_bytree': 0.8067548253988669, 
#          'reg_alpha': 0.06388102700399659, 'reg_lambda': 0.07426015085860142, 
#          'scale_pos_weight': 1.0353953519559222, 'min_split_gain': 0.00010745256855372691, 
#          'min_child_weight': 0.0072034568787835975}

#         lgbm_model = LGBMRegressor(
#             learning_rate=0.02, num_leaves=180, n_estimators=2000, min_child_samples=50,
#             subsample=0.85, colsample_bytree=0.9, reg_alpha=0.02, reg_lambda=0.001,
#             scale_pos_weight=1.2, min_split_gain=1e-4, min_child_weight=0.005,
#             verbose=-1,seed = seed
#               )

        lgbm_model = LGBMRegressor(
                learning_rate=0.0304874420014571, num_leaves=176, n_estimators=1724,
                min_child_samples=12,
                subsample=0.8996187429841459, colsample_bytree=0.8067548253988669,
                reg_alpha=0.06388102700399659, reg_lambda=0.07426015085860142,
                scale_pos_weight=1.0353953519559222, min_split_gain=0.00010745256855372691
                , min_child_weight=0.0072034568787835975,
                verbose=-1,seed = seed
                  )
          ##模型权重不要
        voting_regressor = sklearn.ensemble.VotingRegressor(
            estimators=[('CAT', cat_model), ('Light', lgbm_model)],
            weights=[0.588196, 0.411804]
               )
#         voting_regressor = sklearn.ensemble.VotingRegressor(
#             estimators=[('CAT', cat_model), ('Light', lgbm_model)]
#                )
#         return lgbm_model ##只用lgbm_model测试
#         return cat_model ##只用cat_model测试
        return cat_model
        
#         lgbm_model = LGBMRegressor(verbose = -100, verbose_eval = -1,              
#                          n_estimators = 750,
#                          learning_rate = 0.1,
#                         reg_alpha = 0.05,
#                         reg_lambda = 0.05,
#                         num_leaves = 64,
#                                  seed = seed
#                                 )


#         lgbm_model2 = LGBMRegressor(verbose = -100, verbose_eval = -1,              
#                          n_estimators = 750,
#                          learning_rate = 0.05,
#                         reg_alpha = 0.1,
#                         reg_lambda = 0.1,
#                         num_leaves = 19,
#                                   seed = seed
#                                 )

#         hist_model = sklearn.ensemble.HistGradientBoostingRegressor(max_iter = 500,
#                          learning_rate = 0.1,
#                         max_leaf_nodes = 15,
#                         max_depth = None,
#         categorical_features=None,
#                        random_state = seed)

#         hist_model2 = sklearn.ensemble.HistGradientBoostingRegressor(max_iter = 500,
#                          learning_rate = 0.1,
#                         max_depth = None,
#         categorical_features=None,
#                                 random_state = seed)

#         return sklearn.ensemble.VotingRegressor([("lgbm", lgbm_model), ("hist", hist_model), 
#                                                  ("lgbm2", lgbm_model2), ("hist2", hist_model2)])

    def get_trian_data(self,):
        self.train=self.data_deal(self.train,mode='train')
        return self.train
        
    
    def train_model(self,):
        self.train=self.data_deal(self.train,mode='train')
#         object_columns = self.train.select_dtypes(include=['object']).columns.tolist()
#         print( self.train.dtypes)
#         print(self.train.head)
#         return;

        model = self.make_model()
        X=self.train.drop([self.target,'GameRulesetName'],axis=1)
        X_scaled= self.scaler.fit_transform(X)
        # 将 numpy 数组转换回 pandas DataFrame
        X = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)
        
        GameRulesetName=self.train['GameRulesetName']
        y=self.train[self.target]
    
#         X=X.to_pandas()
#         y=y.to_pandas()
#         X=self.train.drop([],axis=1)
#         sgkf = KFold(n_splits=self.num_folds,random_state=2024,shuffle=True)
        sgkf = GroupKFold(n_splits=self.num_folds)
#         split_list = list(gkf.split(df_train, groups=df_train[groups_col]))

#         for fold, (train_index, valid_index) in (enumerate(sgkf.split(X,y))):
#         print("=======XTYPE========",type(X))
        for fold, (train_index, valid_index) in (enumerate(sgkf.split(X,y,GameRulesetName))):
                print(f"fold:{fold}")
                X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
                y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
#                 print("========valid_index====",train_index,valid_index)
                model.fit(X_train, y_train)
                y_val_pred = model.predict(X_valid)
                print(f"RMSE:{self.RMSE(y_valid,y_val_pred)}")
                self.models.append(model)
                del X_train,X_valid,y_train,y_valid
                gc.collect()
        
        
    def predict(self,test):
        test=self.data_deal(test,mode='test')
        test=test.drop(["Id",'GameRulesetName'],axis=1)
        X_scaled = self.scaler.transform(test)
         # 将 numpy 数组转换回 pandas DataFrame
        test = pd.DataFrame(X_scaled, index=test.index, columns=test.columns)
#         test=test.drop(['GameRulesetName'],axis=1)
#         test=test.to_pandas()
        if len(self.models):#如果有模型
            return np.mean([np.clip(model.predict(test),-1,1) for model in self.models],axis=0)

        

In [ ]:
# preprocessor = Preprocessor(train=train.to_pandas())
# traindata =preprocessor.data_deal(train=train.to_pandas(),mode='train')
# preprocessor.train_model()
# preprocessor.predict(test.to_pandas())
# print(preprocessor.one_hot_str_col)
# print(preprocessor.obj_cols)
# print(preprocessor.preprocessor)
     

In [ ]:
# preprocessor = Preprocessor(train=train.to_pandas())
# train_data = preprocessor.get_trian_data()
# print(train_data)

In [ ]:

# X=train_data.drop([preprocessor.target,"GameRulesetName"],axis=1)
# X_train= preprocessor.scaler.fit_transform(X)
# #         GameRulesetName=self.train['GameRulesetName']
# y_train=train_data[preprocessor.target]

In [ ]:
# preprocessor = Preprocessor(train=train.to_pandas())
# preprocessor.train_model()

GridSearchCV 选择模型参数 

In [ ]:


# #  lgbm_model = LGBMRegressor(
# #             learning_rate=0.02, num_leaves=180, n_estimators=2000, min_child_samples=50,
# #             subsample=0.85, colsample_bytree=0.9, reg_alpha=0.02, reg_lambda=0.001,
# #             scale_pos_weight=1.2, min_split_gain=1e-4, min_child_weight=0.005,
# #             verbose=-1,seed = seed
# #               )

# ##############模型参数选择：3*4*4*3*4*3*4*3*3*3*3 次数太多
# print("================start===============")
# param_grid_lgbm = {
#     'learning_rate': [0.01, 0.02, 0.04],
#     'num_leaves': [64, 128,180, 256],
#     'n_estimators': [500, 1000, 1500, 2000],
#     'min_child_samples': [10, 20, 50],
#     'subsample': [0.8,0.85, 0.9, 1.0],
#     'colsample_bytree': [0.8, 0.9, 1.0],
#     'reg_alpha': [0.01,0.02, 0.05, 0.1],
#     'reg_lambda': [0.01, 0.05, 0.1],
#     'scale_pos_weight': [1.0, 1.2, 1.5],
#     'min_split_gain': [0.0, 1e-4, 1e-3],
#     'min_child_weight': [0.001, 0.005, 0.01],
#     'verbose': [-1],
#     'random_state': [preprocessor.seed]
# }


# grid_search_lgbm = GridSearchCV(estimator=LGBMRegressor(), param_grid=param_grid_lgbm, cv=5, 
#                                 scoring='neg_root_mean_squared_error', n_jobs=-1)
# grid_search_lgbm.fit(X_train, y_train)

# print(f"LGBM Best Parameters: {grid_search_lgbm.best_params_}")
# print(f"LGBM Best Score: {-grid_search_lgbm.best_score_:.4f}")



# 用optuna选择模型参数

In [ ]:
# #######33用optu最优。
# from sklearn.model_selection import cross_val_score
# cat_model = CatBoostRegressor(
#         iterations=1500, learning_rate=0.04, depth=9, l2_leaf_reg=0.0005,
#         min_data_in_leaf=25, bagging_temperature=0.04,
#         grow_policy='SymmetricTree', verbose=0,
#             random_state =  preprocessor.seed,
#         )

# scores = cross_val_score(cat_model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
# mean_score = np.mean(scores)
# print("================scores===============",mean_score)
# ================scores=============== -0.4600240067815641
# # import optuna
# # from lightgbm import LGBMRegressor
# from sklearn.model_selection import cross_val_score
# print("================start===============")
# def objective(trial):
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate',0.01, 0.04),
#         'num_leaves': trial.suggest_int('num_leaves', 64,  256),
#         'n_estimators': trial.suggest_int('n_estimators', 500,  2000),
#         'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
#         'subsample': trial.suggest_float('subsample',0.8, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree',0.8,  1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha',0.01,0.1),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 0.1),
#         'scale_pos_weight': trial.suggest_float('scale_pos_weight',1.0,  1.5),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0.0,  1e-3),
#         'min_child_weight': trial.suggest_float('min_child_weight', 0.001,  0.01),
#         'verbose': -1,
#         'random_state': preprocessor.seed,
# #          'device': 'gpu',  # 使用 GPU
# #         'gpu_platform_id': 0,  # GPU 平台 ID，通常为 0
# #         'gpu_device_id': 0,  # GPU 设备 ID，通常为 0
#     }
    
#     model = LGBMRegressor(**params, device='gpu')
#     scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
#     return scores.mean()

# # 创建 Optuna 研究
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # 输出最佳参数和最佳得分
# print(f"LGBM Best Parameters: {study.best_params}")
# print(f"LGBM Best Score: {-study.best_value:.4f}")



In [ ]:
##查看模型重要特征
# 训练模型
# cat_model.fit(X_train, y_train)



In [ ]:
# X.columns

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# # 获取特征重要性
# feature_importances = cat_model.feature_importances_

# # 将特征重要性转换为 DataFrame
# feature_importance_df = pd.DataFrame({
#     'Feature': X.columns,
#     'Importance': feature_importances
# })

# # 按重要性排序
# feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# # 设置图形大小
# fig, axs = plt.subplots(1, 1, figsize=(10, 100))

# # 设置调色板
# palette = sns.color_palette("RdYlGn_r", len(feature_importance_df))

# # 绘制特征重要性的条形图
# sns.barplot(y='Feature', x='Importance', data=feature_importance_df, orient='h', palette=palette)

# # 调整布局
# plt.tight_layout()

# # 显示图形
# plt.show()

In [ ]:
# from sklearn.model_selection import train_test_split, cross_val_score

# # cat_model = CatBoostRegressor(
# #         iterations=1500, learning_rate=0.04, depth=9, l2_leaf_reg=0.0005,
# #         min_data_in_leaf=25, bagging_temperature=0.04,
# #         grow_policy='SymmetricTree', verbose=0,task_type='GPU',
# #             random_state = seed
# #         )

# def objective(trial):
#     # 定义要优化的超参数范围
#     params = {
#         'iterations': trial.suggest_int('iterations', 500, 2000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 0.1, log=True),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 1.0, log=True),
#         'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree']),
#         'random_state': preprocessor.seed,
    
#         'verbose': 0
#     }

#    # 创建模型
#     model = CatBoostRegressor(**params)
#     # 使用交叉验证评估模型
#     scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
#     mean_score = np.mean(scores)

#     retu
# # 创建 Optuna 研究对象
# study = optuna.create_study(direction='maximize')  # 注意这里的方向是 maximize

# # 运行优化
# study.optimize(objective, n_trials=100)

# # 输出最佳参数
# print("Best parameters:", study.best_params)
# print("Best score:", -study.best_value)  # 转换回正的均方误差

In [ ]:
counter = 0
preprocessor = Preprocessor(train=train.to_pandas())
def predict(test, submission):
    global counter
    if counter == 0:
        # Perform any additional slow steps in the first call to `predict`
        preprocessor.train_model()
    counter += 1    
   
    return submission.with_columns(pl.Series('utility_agent1', preprocessor.predict(test.to_pandas())))

# 提交结果 

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )